## What is an Autoencoder?

Target audiences: Development team and Data science team.

Within artificial neural networks, we can find different types of architectures.  Among theseis the autoencoder neural network.  Autoencoders are an example of self-supervised learn-ing (self-supervised learning is unsupervised learning where data provides the supervision).An autoencoder applies backpropagation.  It takes the input and maps it to a compressedrepresentation  via  an  encoder  and  then  decodes  it  back  to  an  output.   This  output  willhave the same dimensions as the input and will be a reconstruction of it.

An  autoencoder  will  take  the  input,  try  to  encode  features  from  the  data,  compress  thedata  and  from  this  compressed  representation  then  give  an  output  that  will  be  a  recon-struction  of  the  original  input.   For  example,  if  you  input  the  image  of  a  person,  you  willobtain a reconstruction of that same image.

In this notebook we will create a basic autoencoder using R and the MNIST dataset.

To do that, we need to first import the following libraries:

In [ ]:
library(tensorflow)
library(keras)
library(tidyr)
library(ggplot2)
library(dplyr)

We will import the data from the MNIST dataset

In [ ]:
mnist <- dataset_mnist()
x_train <- mnist$train$x/255
x_test <- mnist$test$x/255
x_train <- array_reshape(x_train, c(nrow(x_train), original_dim), order = "F")
x_test <- array_reshape(x_test, c(nrow(x_test), original_dim), order = "F")

We will define the following parameters. These are going to be the dimensions of the different layers.

In [3]:
batch_size       <- 100L
original_dim     <- 784L
latent_dim       <- 2L
intermediate_dim <- 256L

We will now define the model:
- x is the input layer
- h is the hidden layer
- z is the latent space representation

In [4]:
x <- layer_input(shape = c(original_dim))
h <- layer_dense(x, intermediate_dim, activation = "relu")
z <- layer_dense(h, latent_dim)

Encoder: a network that compresses the input into a latent space representation 

In [5]:
encoder <- keras_model(x, z)

Decoder: a network that reconstructs the input from its compressed representation

In [6]:
decoder_h <- layer_dense(units = intermediate_dim, activation = "relu")
decoder_mean <- layer_dense(units = original_dim, activation = "sigmoid")
h_decoded <- decoder_h(z)
x_decoded_mean <- decoder_mean(h_decoded)

We put together the autoencoder and compile it

In [8]:
autoencoder <- keras_model(x, x_decoded_mean)
autoencoder %>% compile(optimizer = "rmsprop", loss = 'categorical_crossentropy')

Our model is ready for training

In [ ]:
autoencoder %>% fit(
  x_train, x_train, 
  shuffle = TRUE, 
  epochs = 50, 
  validation_data = list(x_test, x_test)
)